# Starbucks Customer Behaviour (Data Cleaning, Formatting & Processing)

This notebook has been created to merge, clean & process the data into a format useful for modelling and anaylsis. It follows on from the initial analysis done to gain a better understanding of the data. It has also been used to split the input data into the demographics that will be used to decided how the marketing campagins should be judged.

### Imports

In [1]:
# import general functions
import pandas as pd
import numpy as np
import json

### Functions

In [2]:
def clean_transcript_data(data):
    """
    this process cleans the values column and formats the transcript data
    """
    # creates a column for the type of interaction   
    data['interaction_value'] = [list(x.keys())[0] for x in data['value']]
    
    # creates a column related to the value amount or id    
    data['id'] = [list(x.values())[0] for x in data['value']]
    
    # drops the value column
    data = data.drop(columns=['value'])
    
    # cleans the interaction type column so offer id is consistent
    data['interaction_value'] = [x.replace('offer id','offer_id') for x in data['interaction_value']]
    
    # split out interaction_type
    temp_df = pd.get_dummies(data['interaction_value'])

    # combine the dataframes
    data = pd.concat([temp_df, data], axis=1, sort=True)
    
    # split out event
    temp_df = pd.get_dummies(data['event'])

    # combine the dataframes
    data = pd.concat([temp_df, data], axis=1, sort=True)

    # drop the original columns
    data = data.drop(columns=['interaction_value','event'])
    
    return data # returns the clean transcript data


def clean_profile_data(data):
    """
    this process clean age, income and became_member_on columns in the profile data
    """
    # rename the column 'id' to person
    data.columns = ['age','member joined','gender','person' ,'income']
    
    # replace 118 in the age column with a zero indicating no age 
    # it might be worth looking at this a seperate group of users later on
    data['age'] = data['age'].replace(118,0)

    # update the became_member_on column to a datetime format
    data['member joined'] = pd.to_datetime(data['member joined'], format='%Y%m%d')
    
    # replace the NaN's in the income
    data['income'] = data['income'].fillna(0)
    
    # replace M, F, O and None types to get the 4 groups of customers
    data['gender'] = data['gender'].replace('M','male')
    data['gender'] = data['gender'].replace('F','female')
    data['gender'] = data['gender'].replace('O','other')
    data['gender'] = data['gender'].fillna('unknown gender')

    # split the column into seperate columns
    temp_df = pd.get_dummies(data['gender'])

    # combine the dataframes
    data = pd.concat([temp_df, data], axis=1, sort=True)

    # drop the original column
    data = data.drop(columns=['gender'])

    return data

def clean_portfolio_data(data):
    """
    this process has been created to clean columns in the profile data
    """
    # splits the channels column into seperate columns
    # creates temporary dataframes and lists  
    temp_df = pd.DataFrame(columns=['web', 'email', 'mobile','social'])
    temp_list = []

    # loop through the rows and attach the values to a dic   
    for index, row in data.iterrows():
        for value in row['channels']:
             temp_list.append({'index': index, 'value':value})

    # change the list into dataframe
    temp_df = temp_df.append(temp_list, ignore_index=False, sort=True)
    temp_df = temp_df.groupby('index')['value'].value_counts()
    temp_df = temp_df.unstack(level=-1).fillna(0)
    
    # combine the dataframes
    data = pd.concat([temp_df, data], axis=1, sort=True)
    
    # split the column into seperate columns
    temp_df = pd.get_dummies(data['offer_type'])

    # combine the dataframes
    data = pd.concat([temp_df, data], axis=1, sort=True)

    # drop the original columns
    data = data.drop(columns=['offer_type','channels'])
    
    return data

### Global Variables

In [3]:
# read in the different datasources
portfolio_df = pd.read_json('data/portfolio.json', lines=True)
profile_df = pd.read_json('data/profile.json', lines=True)
transcript_df = pd.read_json('data/transcript.json', lines=True)

### Initial Processing

Below I will run some initial processing of the three input datasets. Details on what these datasets contain can be found in the 'Customer Behaviour notebook' which was ran as a prerequisite to this notebook. 

The intital processing has been ran to split out an catagorical columns so that they have dummies of 1's and 0's depending on if they are true or not. It has cleaned & reformatted some of the data in date columns, replaced NaN vaues as 0 and updated column names so they are consistent across each of the dataframes. The functions can be found above or in the .py file containing all functions that will be used in  future notebooks.

In [4]:
# run the initial cleaning on each dataset
clean_port_df = clean_portfolio_data(portfolio_df)
clean_port_df.head()

,bogo,discount,informational,email,mobile,social,web,difficulty,duration,id,reward
0.0,1,0,0,1.0,1.0,1.0,0.0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,10
1.0,1,0,0,1.0,1.0,1.0,1.0,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,10
2.0,0,0,1,1.0,1.0,0.0,1.0,0,4,3f207df678b143eea3cee63160fa8bed,0
3.0,1,0,0,1.0,1.0,0.0,1.0,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,5
4.0,0,1,0,1.0,0.0,0.0,1.0,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,5


In [5]:
clean_prof_df = clean_profile_data(profile_df)
clean_prof_df.head()

,female,male,other,unknown gender,age,member joined,person,income
0,0,0,0,1,0,2017-02-12,68be06ca386d4c31939f3a4f0e3dd783,0.0
1,1,0,0,0,55,2017-07-15,0610b486422d4921ae7d2bf64640c50b,112000.0
2,0,0,0,1,0,2018-07-12,38fe809add3b4fcf9315a9694bb96ff5,0.0
3,1,0,0,0,75,2017-05-09,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,0,0,0,1,0,2017-08-04,a03223e636434f42ac4c3df47e8bac43,0.0


In [6]:
clean_trans_df = clean_transcript_data(transcript_df)
clean_trans_df.head()

,offer completed,offer received,offer viewed,transaction,amount,offer_id,person,time,id
0,0,1,0,0,0,1,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0,1,0,0,0,1,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,0,1,0,0,0,1,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,0,1,0,0,0,1,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,0,1,0,0,0,1,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


Now that some intital cleaning has been performed we can look further into what inputs we will need for future notebooks including datasets that contain all offer related transactions, one's that contain any unifulenced transactions and a dataset containing all demographic information.

### Offer Influence

The first thing we probably need to decide is how we will determine when an offer has influnced a purchase. We have three different types of offers that need to be handled slightly differently:

 - BOGO (Buy One Get One Free): We can condsider a user has been influneced by this offer when they have viewed the   offer and they have completed the offer at any point throughout the offer period.
 
 
 - Discount: Again we can consider a user has been influenced if they have viewed the offer and completed it in the offer period. We can also consider that they have been influenced on any purchases between viewing and completing the offer as it may be an accumulation offer (e.g. spend more than 10 dollars between a certain period).

- Informational: These are the most difficult to assess in any situation as it's difficult to measure what influence they have had. For this offer type we'll assume that a user was under the influence for 7 days after seeing the offer. Any purchase in that period that also completed the offer will be treated as being influenced.

I'm going to assume if an offer was not completed then a user was not influenced by the offer. Obviously a user could have tried to complete a discount offer but failed but this is difficult to tell from the data provided here. And if the influence had been large enough they would have finished completing the offer.

For the first step I will therefore split out the transactions from the offer data:

In [25]:
# split out the all the transactions
transactions_df = transactions(clean_trans_df)
transactions_df.shape

def transactions(data):
    """
    returns all the transactions from the transcript dataframe
    """
    transactions_df = data[data['transaction'] == 1]
    transactions_df = transactions_df[['person','time','id']]
    transactions_df.columns = ['person','transaction_time','spend']
    
    return transactions_df

Now we can split out the offers and process them seperately based on the criteria above:

In [30]:
def offers(transcript_data, portfolio_data):
    """
    returns all of the offers that were received/viewed/completed combined with portfolio data
    """
    # keep only the recived offers
    received_offer = transcript_data[transcript_data['offer received'] == 1]
    received_offer = received_offer[['offer received','person', 'time', 'id']]
    received_offer.columns = ['offer received','person', 'time_received', 'id_offer']    
    
    # keep only the viewed offers
    veiwed_offer = transcript_data[transcript_data['offer viewed'] == 1]
    veiwed_offer = veiwed_offer[['offer viewed','person', 'time', 'id']]
    veiwed_offer.columns = ['offer viewed','person', 'time_viewed', 'id_offer']
    
    # keep all the offers completed data as informational campaigns don't have a completed flag
    completed_offer = transcript_data
    completed_offer = completed_offer[['offer completed','person', 'time', 'id']]
    completed_offer.columns = ['offer completed','person', 'time_completed', 'id_offer']
    
    # merge the offers data into one dataframe based on id and person
    merged_veiws = received_offer.merge(veiwed_offer, on=['person','id_offer']) 
    merged_completed = merged_veiws.merge(completed_offer, on=['person','id_offer']) 
    
    # drop anywhere the offer was recived after being viewed (not useful)
    merged_completed = merged_completed[merged_completed['time_viewed'] > 
                                        merged_completed['time_received']]
    
    # merges all of the offer data with info in the portfolio data
    portfolio_data = portfolio_data.rename(columns = {'id':'id_offer'})
    offers = merged_completed.merge(portfolio_data, on=['id_offer'])
    
    # change duration time to hours
    offers['duration'] = offers['duration']*24
    
    # update informational offers to impact users 7 days after  
    offers.duration = np.where(offers.informational.eq(1), 
                               (offers.time_viewed +(7*24)), 
                               offers.duration)
    
    return offers

def influenced_bogo(transcript_data, portfolio_data):
    """
    this function has been created to keep only BOGO offers that influenced a purchase
    """
    # gets all of the offers that were received/viewed/completed formatted together
    offer_data = offers(transcript_data, portfolio_data)
    
    # select only the bogo offers and have been completed
    bogo_offers = offer_data[(offer_data['bogo'] == 1) & (offer_data['offer completed'] == 1)]
    
    # removes any that were completed prior to being viewed
    bogo_offers = bogo_offers[bogo_offers['time_completed'] >= bogo_offers['time_viewed']]
    
    # removes offers that were completed outside of the offer timeframe (indicating it was a second offer)
    bogo_offers = bogo_offers[bogo_offers['duration'] <= (bogo_offers['time_completed'] - 
                                                          bogo_offers['time_received'])]

    # creates the transaction data
    transactions_data = transactions(transcript_data)
    
    # merge the offers and transactions
    transactions_bogo = transactions_data.merge(bogo_offers, on=['person'])
    
    # filter the tansactions keeping ones that occured at same time as the offer was complete
    transactions_bogo = transactions_bogo[transactions_bogo['transaction_time'] == 
                                          transactions_bogo['time_completed']]
    
    # remove any repeat transactions
    transactions_bogo = transactions_bogo.drop_duplicates(subset=['person','transaction_time','spend'], keep="first")
    
    return transactions_bogo

def influenced_discount(transcript_data, portfolio_data):
    """
    this function has been created to keep only discount offers that influenced a purchase
    """
    # gets all of the offers that were received/viewed/completed formatted together
    offer_data = offers(transcript_data, portfolio_data)
    
    # select only the discuont offers and have been completed
    discount_offers = offer_data[(offer_data['discount'] == 1) & (offer_data['offer completed'] == 1)]
    
    # removes offers that were completed outside of the timeframe (indicating it was a second offer)
    discount_offers = discount_offers[discount_offers['duration'] <= (discount_offers['time_completed'] - 
                                                                      discount_offers['time_received'])]

    # creates the transaction data
    transactions_data = transactions(transcript_data)
    
    # merge the offers and transactions
    transactions_discount = transactions_data.merge(discount_offers, on=['person'])
    
    # filter the tansactions keeping the ones after the offer was viewed by before it was completed
    transactions_discount = transactions_discount[(transactions_discount['transaction_time'] >= transactions_discount['time_viewed']) &
                                                 (transactions_discount['transaction_time'] <= transactions_discount['time_completed'])]
    
    # remove any repeat transactions
    transactions_discount = transactions_discount.drop_duplicates(subset=['person','transaction_time','spend'], keep="first")
    
    return transactions_discount

def influenced_informational(transcript_data, portfolio_data):
    """
    this function has been created to keep only informational offers that influenced a purchase
    """
    # gets all of the offers that were received/viewed/completed formatted together
    offer_data = offers(transcript_data, portfolio_data)
    
    # select only the informational offers
    info_offers = offer_data[(offer_data['informational'] == 1)]

    # creates the transaction data
    transactions_data = transactions(transcript_data)
    
    # merge the offers and transactions
    transactions_info = transactions_data.merge(info_offers, on=['person'])
    
    # filter the tansactions keeping the ones after the offer was viewed by before it was completed
    transactions_info = transactions_info[(transactions_info['transaction_time'] >= transactions_info['time_viewed'])]
    
    # removes transactions that happened outside of the 7 day timeframe
    transactions_info = transactions_info[transactions_info['duration'] <= (transactions_info['transaction_time'] - 
                                                                            transactions_info['time_viewed'])]
    
    # remove any repeat transactions
    transactions_info = transactions_info.drop_duplicates(subset=['person','transaction_time','spend'], keep="first")
    
    return transactions_info

In [31]:
inf_discount = influenced_discount(clean_trans_df, clean_port_df)
print(inf_discount.shape)
inf_discount.head()

(9835, 20)


,person,transaction_time,spend,offer received,time_received,id_offer,offer viewed,time_viewed,offer completed,time_completed,bogo,discount,informational,email,mobile,social,web,difficulty,duration,reward
4,629fc02d56414d91bca360decdfa9288,666,21.36,1,408,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,462,1,666,0,1,0,1.0,0.0,0.0,1.0,20,240,5
8,bb0f25e23a4c4de6a645527c275cd594,162,21.62,1,0,2298d6c36e964ae4a3e7e9706d1fb8c2,1,96,1,654,0,1,0,1.0,1.0,1.0,1.0,7,168,3
10,bb0f25e23a4c4de6a645527c275cd594,264,35.95,1,0,2298d6c36e964ae4a3e7e9706d1fb8c2,1,96,1,654,0,1,0,1.0,1.0,1.0,1.0,7,168,3
12,bb0f25e23a4c4de6a645527c275cd594,450,25.18,1,0,2298d6c36e964ae4a3e7e9706d1fb8c2,1,96,1,654,0,1,0,1.0,1.0,1.0,1.0,7,168,3
14,bb0f25e23a4c4de6a645527c275cd594,474,24.4,1,0,2298d6c36e964ae4a3e7e9706d1fb8c2,1,96,1,654,0,1,0,1.0,1.0,1.0,1.0,7,168,3


In [32]:
inf_bogo = influenced_bogo(clean_trans_df, clean_port_df)
print(inf_bogo.shape)
inf_bogo.head()

(2235, 20)


,person,transaction_time,spend,offer received,time_received,id_offer,offer viewed,time_viewed,offer completed,time_completed,bogo,discount,informational,email,mobile,social,web,difficulty,duration,reward
5,3e621194f72e40d7a0b695ee9b7c38b7,396,14.9,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0,1,348,1,396,1,0,0,1.0,1.0,1.0,1.0,10,120,10
9,99297ea01107436fa8c2e2bc86f55d89,372,8.97,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,330,1,372,1,0,0,1.0,1.0,0.0,1.0,5,168,5
24,afce4cf8194f4e90a3e92da941a23601,534,15.19,1,0,ae264e3637204a6fb9bb56bc8210ddfd,1,18,1,534,1,0,0,1.0,1.0,1.0,0.0,10,168,10
49,c3321bc76743445c9103eef0ce9d833b,552,46.14,1,336,4d5c57ea9a6940dd891ad53e9dbe8da0,1,360,1,552,1,0,0,1.0,1.0,1.0,1.0,10,120,10
66,5152fa6375184287b06e2fd0d5abed34,582,11.79,1,0,f19421c1d4aa40978ebb69ca19b0e20d,1,534,1,582,1,0,0,1.0,1.0,1.0,1.0,5,120,5


In [33]:
inf_informational = influenced_informational(clean_trans_df, clean_port_df)
print(inf_informational.shape)
inf_informational.head()

(11477, 20)


,person,transaction_time,spend,offer received,time_received,id_offer,offer viewed,time_viewed,offer completed,time_completed,bogo,discount,informational,email,mobile,social,web,difficulty,duration,reward
36,b2f1cd155b864803ad8334cdf13c4bd2,384,29.95,1,0,5a8bc65990b245e5a138643cd4eb9837,1,66,0,0,0,0,1,1.0,1.0,1.0,0.0,0,234,0
40,b2f1cd155b864803ad8334cdf13c4bd2,468,24.55,1,0,5a8bc65990b245e5a138643cd4eb9837,1,66,0,0,0,0,1,1.0,1.0,1.0,0.0,0,234,0
42,b2f1cd155b864803ad8334cdf13c4bd2,570,28.1,1,168,3f207df678b143eea3cee63160fa8bed,1,198,0,168,0,0,1,1.0,1.0,0.0,1.0,0,366,0
270,fe97aa22dd3e48c8b143116a8403dd52,624,14.77,1,168,3f207df678b143eea3cee63160fa8bed,1,198,0,168,0,0,1,1.0,1.0,0.0,1.0,0,366,0
302,fe97aa22dd3e48c8b143116a8403dd52,642,19.26,1,168,3f207df678b143eea3cee63160fa8bed,1,198,0,168,0,0,1,1.0,1.0,0.0,1.0,0,366,0


It's possible that the some of the transactions above would have been brought regardless of if the user had seen the ads esspecially in the case of the informational offers. In the next part of this processing notebook we will calculate how much users spend on average over time and per transaction. This will help to further process the above transactions and also decided help to decide how to split the different demographic groups.

### Normal Customer Behaviour

In [42]:
# Now that we have all the influenced transactions we can find the ones not influenced by offers
def norm_transactions(transcript_data, portfolio_data):
    """
    produces all the transactions that weren't influenced by offers
    """
    # creates the transaction data
    transactions_data = transactions(transcript_data)
    
    # all offer affected transactions
    inf_discount = influenced_discount(clean_trans_df, clean_port_df)
    inf_bogo = influenced_bogo(clean_trans_df, clean_port_df)
    inf_informational = influenced_informational(clean_trans_df, clean_port_df)
    
    # combine all the influenced transcations    
    inf_trans = inf_informational.append(inf_discount.append(inf_bogo))
    
    # drop to have the same columns as all transactions    
    inf_trans = inf_trans[['person', 'transaction_time', 'spend']]
    
    # remove offer related transactions
    norm_trans = pd.concat([transactions_data, inf_trans]).drop_duplicates(keep=False)
    
    return norm_trans

In [45]:
uninflunced_trans = norm_transactions(clean_trans_df, clean_port_df)
uninflunced_trans.head()

,person,transaction_time,spend
12654,02c083884c7d45b39cc68e1314fec56c,0,0.83
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56
12659,54890f68699049c2a04d415abc25e717,0,13.23
12670,b2f1cd155b864803ad8334cdf13c4bd2,0,19.51
12671,fe97aa22dd3e48c8b143116a8403dd52,0,18.97


Now that we have all the normal transactions we can create useful metrics based on the users normal behaviour. These could be 

In [ ]:
# Total value of transactions made by a user
prof_trans = pd.DataFrame(uninflunced_trans.groupby('person')['interaction_value'].sum().values, 
                          index=uninflunced_trans.groupby('person')['interaction_value'].sum().index,
                          columns=['total sales'])

# Number of transactions made by a user
prof_trans['number of sales'] = uninflunced_trans.groupby('person')['interaction_value'].count().values

# Average value of transactions made by a user
prof_trans['mean spend'] = (prof_trans['total sales']/prof_trans['number of sales'])

# set person as index
clean_prof_df.set_index('person', inplace = True)

# combine the two datasets
user_data = pd.concat([clean_prof_df, prof_trans], axis=1, sort=True).fillna(0)

Now we have all the data on our users transactions and the demographics to try to see how we should split up our demographic data. At this point we have ingorned any of the offers influnences on the demographics and that should be fine for this analysis as if some users are more heavily influnced by an offer we want that to stand out in the demographic split e.g. if users between 20-30 spend much more due to offers we want them as a seperate cluster in our customer segmentaion technique.

Now we have our user data and transaction history it might be worth creating an extra column to show how much they've spent/number of transactions a week on average since signing up as this might be more insightful than just looking at total transaction data.

In [ ]:
# I will take the last date in the data as the final time data was collected
final_date = user_data['member joined'].max()

# Then we will create a length in weeks that the user has been a member at starbucks
user_data['membership length'] = [round((final_date - x).days / 7,0) for x in user_data['member joined']]

# This can be used to calculate the average money spent per week
user_data['weekly spend'] = user_data['total sales']/user_data['membership length']

# Drop memeber joined column
user_data = user_data.drop(columns=['member joined'])

In [ ]:
user_data.head()

Now we have the user gender, age, income, lifetime sales, number of sales, averge sale, membership length, sales per week and average number of transactions per week. For the sake of removing other features of the analysis it's worth removing number of sales and total sales so that membership length is only accounted for once.

Now our input data is ready to perform unsupervised learning on to try to determine which demographics we show be splitting the data into. Here we will use the kmeans algorthim which is a clustering algorthim and we will split the data into the number of clusters that

The results above show that splitting the users into 5-10 clusters gives the largest amount of information gain for the smallest number of splits in the data. Below I have split the data into 8 seperate clusters and we can seen what each one of these demographics looks like.

Looking at the clusters the data has been split into above shows there are many interesting features. Consumers with no age or gender information (Cluster 3) spent very little on average and are probably not that interested in using the membership at all. These might be able to be ruled out when judging marketing performance as they bring very little value. They make up around 2,000 users on the app.

Another obvious cluster (6) shows that users with short memberships (around a week) have very large order values this could be the influence of the reward scheme and maybe show users that could be easy to influence into buying more products. However, this is a very small amount of users and so it might not be useful.

One of the intital observations is that overall all of the clusters have a similar average age:

This shows that age isn't a major contributing factor into how much users are paying or how often they go to Starbucks which means it shouldn't be a key focus of our demographic splits. This was surprising as I think the common perception is that Starbucks consumers would be a younger demographic.

One of the biggest impacts seemed to come from gender. Interestingly majority of female consumers have been grouped together in cluster 0:

This shows that female users mostly acted very similarly in terms of buying habits with similar weekly spend and mean spend per shop.

Cluster 5 has mainly very wealthy users who spend a considerable amount when they visit the store. The gender split is almost 50/50 but they are more likely to be older consumers:

Cluster 2 is everyone who identified them selves as other in terms of gender. That leaves 1 and 4 that are the catagories accositated with male consumers only:

Users in cluster 4 appear to have a longer membershop and lower weekly spend. Whereas users in cluster 1 have a shorter membership time bit a higher weekly spend. Given the transaction history and user we can now assign uders to each of the 7 demographic catagories given above and ajust media targeting accordingly to the user in them. 